In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('df4.csv')

### Base Model Building

In [16]:
df = df.drop(columns= ['Unnamed: 0'])

In [17]:
df.head()

,509(a)(3) Type I,509(a)(3) Type II,509(a)(3) Type III functionally integrated,Adult Education Programs and Services,Advocacy and Education,"Animal Rights, Welfare, and Services",Animals,"Arts, Culture, Humanities","Botanical Gardens, Parks, and Nature Centers","Central - the organization is a central type organization (no group exemption) of a National, Regional or Geographic grouping of organizations.",Central - the organization is a parent (group ruling) and is a church or 501(c)(1) organization.,Central - the organization is a parent (group ruling) and is not a church or 501(c)(1) organization.,Children's and Family Services,Church 170(b)(1)(A)(i),Community Development,Community Foundations,Development and Relief Services,"Diseases, Disorders, and Disciplines",Early Childhood Programs and Services,Education,Education Policy and Reform,Environment,Environmental Protection and Conservation,"Food Banks, Food Pantries, and Food Distribution",Governmental unit 170(b)(1)(A)(v),Health,Homeless Services,Hospital or medical research organization 170(b)(1)(A)(iii),Housing and Neighborhood Development,Human Services,Human and Civil Rights,Humanitarian Relief Supplies,"Independent - the organization is an independent organization or an independent auxiliary (i.e., not affiliated with a National, Regional, or Geographic grouping of organizations).","Intermediate - the organization is an intermediate organization (no group exemption) of a National, Regional or Geographic grouping of organizations (such as a state headquarters of a national organization).",International,"International Peace, Security, and Affairs",Jewish Federations,"Libraries, Historical Societies and Landmark Preservation",Medical Research,Midwest,Multipurpose Human Service Organizations,Museums,Non-Medical Science & Technology Research,Northeast,"Organization that normally receives no more than one-third of its support from gross investment income and unrelated business income and at the same time more than one-third of its support from contributions, fees, and gross receipts related to exempt purposes. 509(a)(2)",Organization which operates for benefit of college or university and is owned or operated by a governmental unit 170(b)(1)(A)(iv),Organization which receives a substantial part of its support from a governmental unit or the general public 170(b)(1)(A)(vi),Organizations operated solely for the benefit of and in conjunction with organizations described in 10 through 16 above. 509(a)(3),Patient and Family Support,Performing Arts,Private non-operating foundation,Public Broadcasting and Media,Religion,Religious Activities,Religious Media and Broadcasting,Rescue Missions,Research and Public Policy,Scholarship and Financial Support,School 170(b)(1)(A)(ii),Social Services,Social and Public Policy Research,Southeast,Southwest,Special Education,Subordinate - the organization is a subordinate in a group ruling.,Treatment and Prevention Services,United Ways,West,Wildlife Conservation,"Youth Development, Shelter, and Crisis Services",Youth Education Programs and Services,Zoos and Aquariums,accountability_rating,accountability_score,category_ID,cause_ID,current_rating,current_score,fin_rating,fin_score,org_assets,org_income,zipcode
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,82.0,6.0,29.0,3.0,85.98,4.0,91.70,2032739.0,3364323.0,35209.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,85.0,9.0,26.0,3.0,89.04,4.0,96.10,2602296.0,1930948.0,36801.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
corr=df[features].corr()

kot = corr[corr>=.7]
plt.figure(figsize=(12,8))

sns.heatmap(kot, cmap="Reds")